In [42]:
import pandas as pd

In [73]:
file = '/Users/test/Works/dirsearch/reports/_127.0.0.1_18000/features.csv'
df = pd.read_csv(file)

In [56]:
import identify404

In [125]:
rsp_body_pattern_columns = [name for name in df.columns if name.startswith('c:')]
fdf = df[rsp_body_pattern_columns].div(df.standard_body_length, axis=0)
fdf = pd.concat([fdf,pd.get_dummies(df['status_code'], prefix='code')], axis=1)
# fdf['body_length'] = (df.body_length-df.body_length.mean())/df.body_length.std()
fdf['standard_distance'] = df.body_length - df.standard_body_length
fdf['standard_distance'] = (fdf.standard_distance-fdf.standard_distance.mean())/fdf.standard_distance.std()
fdf['standard_body_length'] = (df.standard_body_length-df.standard_body_length.mean())/df.standard_body_length.std()
# fdf = (fdf-fdf.mean())/fdf.std()
fdf.fillna(0, inplace=True)
labels, results, cluster = identify404.identify_404(fdf.to_numpy())

current_ratio=0 {'count': 1, 'ratio': 0.0008347245409015025, 'success': False}
current_ratio=0.0008347245409015025 {'count': 3, 'ratio': 0.0025041736227045075, 'success': False}
current_ratio=0.00333889816360601 {'count': 6, 'ratio': 0.005008347245409015, 'success': False}
current_ratio=0.008347245409015025 {'count': 82, 'ratio': 0.06844741235392321, 'success': False}
current_ratio=0.07679465776293823 {'count': 1106, 'ratio': 0.9232053422370617, 'success': False}


In [126]:
cluster

{'bestClusters': 5,
 'bestScore': 0.9474965954136929,
 'bestK': 5,
 'labelDescription': {'2': {'count': 1106,
   'ratio': 0.9232053422370617,
   'success': False},
  '3': {'count': 6, 'ratio': 0.005008347245409015, 'success': True},
  '0': {'count': 82, 'ratio': 0.06844741235392321, 'success': True},
  '4': {'count': 1, 'ratio': 0.0008347245409015025, 'success': True},
  '1': {'count': 3, 'ratio': 0.0025041736227045075, 'success': True}}}

In [127]:
df['exists'] = df['status_code']==200
df['labels'] = labels
df['exists2'] = results

In [128]:
df.labels.value_counts()

2    1106
0      82
3       6
1       3
4       1
Name: labels, dtype: int64

In [129]:
df.exists2.value_counts()

False    1106
True       92
Name: exists2, dtype: int64

In [122]:
df[df['exists2']==True]

,status_code,body_length,standard_body_length,c:<,c:>,c:/,c:</,c:/>,c:=/,c:.,...,"c:""","c:,",c:=,c:(,c:,c:;,url,exists,labels,exists2
56,200,8196,8115,0,0,0,0,0,0,5,...,0,9,4,5,0,3,http://127.0.0.1:18000/.DS_Store,True,3,True
79,301,314,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git,False,4,True
80,200,2872,2554,226,226,153,105,0,0,21,...,162,0,88,1,1,17,http://127.0.0.1:18000/.git/,True,4,True
81,200,760,687,61,61,45,28,0,0,11,...,30,0,22,1,1,6,http://127.0.0.1:18000/.git/branches/,True,4,True
82,200,277,240,0,0,11,0,0,0,2,...,4,0,8,0,0,0,http://127.0.0.1:18000/.git/config,True,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,403,301,245,17,17,15,7,1,0,7,...,2,0,0,1,1,0,http://127.0.0.1:18000/server-status/,False,4,True
1068,301,314,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/test,False,4,True
1069,200,1137,1022,91,91,60,42,0,0,14,...,54,0,34,1,1,8,http://127.0.0.1:18000/test/,True,4,True
1148,301,314,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/wiki,False,4,True


In [130]:
# 资产实际存在，但算法认为不存在
df[(df['exists']==True) & (df['exists2']==False)]

,status_code,body_length,standard_body_length,c:<,c:>,c:/,c:</,c:/>,c:=/,c:.,...,"c:""","c:,",c:=,c:(,c:,c:;,url,exists,labels,exists2


In [131]:
# 资产实际不存在，但算法认为存在
df[(df['exists']==False) & (df['exists2']==True)]

,status_code,body_length,standard_body_length,c:<,c:>,c:/,c:</,c:/>,c:=/,c:.,...,"c:""","c:,",c:=,c:(,c:,c:;,url,exists,labels,exists2
79,301,314,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git,False,0,True
91,301,324,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/logs/refs,False,0,True
92,301,330,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/logs/refs/heads,False,0,True
94,301,332,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/logs/refs/remotes,False,0,True
95,301,339,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/logs/refs/remotes/...,False,0,True
100,301,325,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/refs/heads,False,0,True
102,301,327,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/refs/remotes,False,0,True
103,301,334,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/refs/remotes/origin,False,0,True
105,301,324,250,18,18,16,8,0,0,7,...,4,0,1,1,1,0,http://127.0.0.1:18000/.git/refs/tags,False,0,True
116,403,298,245,17,17,15,7,1,0,7,...,2,0,0,1,1,0,http://127.0.0.1:18000/.ht_wsr.txt,False,0,True


In [132]:
df[(df['exists']==False) & (df['exists2']==True)].status_code.value_counts()

403    41
301    14
Name: status_code, dtype: int64

In [100]:
cdf = df.filter(regex='^c:', axis=1)

In [98]:
cdf

,c:<,c:>,c:/,c:</,c:/>,c:=/,c:.,c:',c:[,c:],...,c:{,c:},c::,"c:""","c:,",c:=,c:(,c:,c:;,standard_body_length
0,16,16,16,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,237
1,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234
2,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234
3,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234
4,16,16,13,6,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234
1194,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234
1195,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234
1196,16,16,14,7,0,0,7,0,0,0,...,0,0,0,2,0,0,1,1,0,234


In [101]:
columns = cdf.columns
# cdf['standard_body_length'] = df['standard_body_length']
cdf[columns].div(df.standard_body_length, axis=0)

,c:<,c:>,c:/,c:</,c:/>,c:=/,c:.,c:',c:[,c:],...,c:*,c:{,c:},c::,"c:""","c:,",c:=,c:(,c:,c:;
0,0.067511,0.067511,0.067511,0.029536,0.0,0.0,0.029536,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008439,0.0,0.0,0.004219,0.004219,0.0
1,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0
2,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0
3,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0
4,0.069869,0.069869,0.056769,0.026201,0.0,0.0,0.030568,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008734,0.0,0.0,0.004367,0.004367,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0
1194,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0
1195,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0
1196,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.008547,0.0,0.0,0.004274,0.004274,0.0


In [106]:
rsp_body_pattern_columns = [name for name in df.columns if name.startswith('c:')]
fdf = df[rsp_body_pattern_columns].div(df.standard_body_length, axis=0)
fdf = pd.concat([fdf,pd.get_dummies(df['status_code'], prefix='code')], axis=1)
fdf['body_length'] = (df.body_length-df.body_length.mean())/df.body_length.std()
fdf['standard_body_length'] = (df.standard_body_length-df.standard_body_length.mean())/df.standard_body_length.std()
# fdf = (fdf-fdf.mean())/fdf.std()
fdf.fillna(0, inplace=True)
fdf

,c:<,c:>,c:/,c:</,c:/>,c:=/,c:.,c:',c:[,c:],...,c:=,c:(,c:,c:;,code_200,code_301,code_403,code_404,body_length,standard_body_length
0,0.067511,0.067511,0.067511,0.029536,0.0,0.0,0.029536,0.0,0.0,0.0,...,0.0,0.004219,0.004219,0.0,0,0,0,1,-0.092611,-0.094032
1,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.092094,-0.096343
2,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.091576,-0.096343
3,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.092094,-0.096343
4,0.069869,0.069869,0.056769,0.026201,0.0,0.0,0.030568,0.0,0.0,0.0,...,0.0,0.004367,0.004367,0.0,0,0,0,1,-0.091576,-0.100196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.091059,-0.096343
1194,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.091059,-0.096343
1195,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.091576,-0.096343
1196,0.068376,0.068376,0.059829,0.029915,0.0,0.0,0.029915,0.0,0.0,0.0,...,0.0,0.004274,0.004274,0.0,0,0,0,1,-0.091576,-0.096343
